In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

In [3]:
spark = pyspark.sql.SparkSession.builder.appName('MostPopularHero').getOrCreate()

In [4]:
heroNames = spark.read.csv('old/Marvel-Names.txt', sep='\"')
heroNames = heroNames.selectExpr('_c0 as heroId', '_c1 as heroName')
heroNames.show(5)

+------+--------------------+
|heroId|            heroName|
+------+--------------------+
|    1 |24-HOUR MAN/EMMANUEL|
|    2 |3-D MAN/CHARLES CHAN|
|    3 |    4-D MAN/MERCURIO|
|    4 |             8-BALL/|
|    5 |                   A|
+------+--------------------+
only showing top 5 rows



In [8]:
occurs = spark.read.csv('old/Marvel-Graph.txt', sep='|')
occurs = occurs.withColumnRenamed('_c0', 'occurs')
occursSplit = pyspark.sql.functions.split(occurs.occurs, ' ')
occurs = occurs.withColumn('occurs', occursSplit)
occurs.show(5)
occurs.count()

+--------------------+
|              occurs|
+--------------------+
|[5988, 748, 1722,...|
|[5989, 4080, 4264...|
|[5982, 217, 595, ...|
|[5983, 1165, 3836...|
|[5980, 2731, 3712...|
+--------------------+
only showing top 5 rows



6589

In [9]:
occursExploded = pyspark.sql.functions.explode(occurs.occurs)
occurs = occurs.withColumn('heroId', occursExploded)
occurs = occurs.drop('occurs')
occurs = occurs.where(occurs.heroId != '')
occurs.show(5)

+------+
|heroId|
+------+
|  5988|
|   748|
|  1722|
|  3752|
|  4655|
+------+
only showing top 5 rows



In [10]:
heroOccurs = occurs.groupby('heroId').count()
heroOccurs = heroOccurs.withColumn('heroId', heroOccurs.heroId.cast('integer'))
heroOccurs.show(5)

+------+-----+
|heroId|count|
+------+-----+
|  2069|  264|
|  3210|  119|
|   829|   38|
|  6194|   15|
|  4821|   17|
+------+-----+
only showing top 5 rows



In [11]:
heroNames.join(heroOccurs, 'heroId').orderBy('count', ascending=False).show(5)

+------+--------------------+-----+
|heroId|            heroName|count|
+------+--------------------+-----+
|  859 |     CAPTAIN AMERICA| 1937|
| 5306 |SPIDER-MAN/PETER PAR| 1745|
| 2664 |IRON MAN/TONY STARK | 1532|
| 5716 |THING/BENJAMIN J. GR| 1429|
| 6306 |    WOLVERINE/LOGAN | 1397|
+------+--------------------+-----+
only showing top 5 rows



In [ ]:
spark.stop()